In [2]:
import numpy as np
import numpy.linalg as la

# Q2b,c,

In [31]:
A = np.array([[0, 0, 0, 1],
             [1, 0, 0, 0],
             [0, 0.5, 0, 0],
             [0, 0.5, 1, 0]])

def get_PageRank(A, guess, epsilon = 0.01):
    last_pi = guess / la.norm(guess)
    while True:
        next_pi = A@last_pi
        next_pi = next_pi / la.norm(next_pi)
        if (la.norm(next_pi - last_pi)) > epsilon:
            return next_pi
        last_pi = next_pi


pi = get_PageRank(A, np.array([1, 1, 1, 1]), 0.0001)
print(f"PageRank for 4 pages in Q2b is {pi}")

def get_GooglePageRank(A, a):
    n,_ = A.shape
    u = np.ones(n)
    G = a*A + (1-a)*(1/n)*(u@u.T)
    G = G/G.sum(axis=0)

    return get_PageRank(G, u)

G_pi = 


PageRank for 4 pages in Q2b is [0.47140452 0.47140452 0.23570226 0.70710678]


In [34]:
get_GooglePageRank(A, 0.8)

array([0.49236596, 0.49236596, 0.36927447, 0.61545745])